Import the Topography Data
Firstly, importing the values of latitude, longitude, and topography from the downloaded NetCDF file using the Etopo function.
Input: longitude (lon_area) and latitude (lat_area) based on the selected area, and resolution.
Output: mesh shape longitude (lon), latitude (lat) within the selected area, and topography (topo) based on resolution.
* resolution has to be in degree unit and the highest resolution will be 0.0167 degree

In [72]:
import os

In [73]:
import numpy as np

In [74]:
import import_ipynb 
from netCDF4 import Dataset

In [75]:
def Etopo(lon_area, lat_area, resolution):
  ### Input
  # resolution: resolution of topography for both of longitude and latitude [deg]
  # (Original resolution is 0.0167 deg)
  # lon_area and lat_area: the region of the map which you want like [100, 130], [20, 25]
  ###
  ### Output
  # Mesh type longitude, latitude, and topography data
  ###
  
  # Read NetCDF data
  data = Dataset("ETOPO1_Ice_g_gdal.grd", "r")
  
  # Get data
  lon_range = data.variables['x_range'][:]
  lat_range = data.variables['y_range'][:]
  topo_range = data.variables['z_range'][:]
  spacing = data.variables['spacing'][:]
  dimension = data.variables['dimension'][:]
  z = data.variables['z'][:]
  lon_num = dimension[0]
  lat_num = dimension[1]
  
  # Prepare array
  lon_input = np.zeros(lon_num); lat_input = np.zeros(lat_num)
  for i in range(lon_num):
    lon_input[i] = lon_range[0] + i * spacing[0]
  for i in range(lat_num):
    lat_input[i] = lat_range[0] + i * spacing[1]

  # Create 2D array
  lon, lat = np.meshgrid(lon_input, lat_input)
  
  # Convert 2D array from 1D array for z value
  topo = np.reshape(z, (lat_num, lon_num))
  
  # Skip the data for resolution
  if ((resolution < spacing[0]) | (resolution < spacing[1])):
    print('Set the highest resolution')
  else:
    skip = int(resolution/spacing[0])
    lon = lon[::skip,::skip]
    lat = lat[::skip,::skip]
    topo = topo[::skip,::skip]
    
  topo = topo[::-1]
  
  # Select the range of map
  range1 = np.where((lon>=lon_area[0]) & (lon<=lon_area[1]))
  lon = lon[range1]; lat = lat[range1]; topo = topo[range1]
  range2 = np.where((lat>=lat_area[0]) & (lat<=lat_area[1]))
  lon = lon[range2]; lat = lat[range2]; topo = topo[range2]
  
  # Convert 2D again
  lon_num = len(np.unique(lon))
  lat_num = len(np.unique(lat))
  lon = np.reshape(lon, (lat_num, lon_num))
  lat = np.reshape(lat, (lat_num, lon_num))
  topo = np.reshape(topo, (lat_num, lon_num))
  
  return lon, lat, topo


1–2. Converting to the spherical coordinates
To make the spherical plot using the imported longitude (lon) and latitude (lat) data, we convert these data into the spherical coordinates. The idea is not complicated. Just converting from the orthogonal coordinate to the spherical coordinate using the basic math functional form.

In [76]:
def degree2radians(degree):
  # convert degrees to radians
  return degree*np.pi/180
  
def mapping_map_to_sphere(lon, lat, radius=1):
  # this function maps the points of coords (lon, lat) to points onto the sphere of radius radius
  lon=np.array(lon, dtype=np.float64)
  lat=np.array(lat, dtype=np.float64)
  lon=degree2radians(lon)
  lat=degree2radians(lat)
  xs=radius*np.cos(lon)*np.cos(lat)
  ys=radius*np.sin(lon)*np.cos(lat)
  zs=radius*np.sin(lat)
  return xs, ys, zs

1–3. Plot the Interactive Globe using Plotly
Here we plot the interactive Globe using the data of latitude, longitude described as the spherical coordinated, and topography.
Firstly, we call the Etopo function defined in Step. 1 to read the array of longitude (lon_topo), latitude (lat_topo), and topography (topo) covering the whole globe. If the resolution is too high, the size of data becomes huge due to the 3-dimensional data. Here we adopt 0.8 degrees as the value of resolution.


In [77]:
# Import topography data
# Select the area you want
resolution = 0.8
lon_area = [-180., 180.]
lat_area = [-90., 90.]
# Get mesh-shape topography data
lon_topo, lat_topo, topo = Etopo(lon_area, lat_area, resolution)

And then, converting to the spherical coordinates using the created function of mapping_map_to_sphere

In [78]:

xs , ys , zs  =  mapping_map_to_sphere ( lon_topo , lat_topo )

We also need to define the color scale of the topography we plot. The color scale in this example is defined as follows:

In [79]:
Ctopo = [[0, 'rgb(0, 0, 70)'],[0.2, 'rgb(0,90,150)'], 
          [0.4, 'rgb(150,180,230)'], [0.5, 'rgb(210,230,250)'],
          [0.50001, 'rgb(0,120,0)'], [0.57, 'rgb(220,180,130)'], 
          [0.65, 'rgb(120,100,0)'], [0.75, 'rgb(80,70,0)'], 
          [0.9, 'rgb(200,200,200)'], [1.0, 'rgb(255,255,255)']]
cmin = -8000
cmax = 8000

We prepare the plot using Plotly as follows:

In [80]:
topo_sphere=dict(type='surface',
  x=xs,
  y=ys,
  z=zs,
  colorscale=Ctopo,
  surfacecolor=topo,
  cmin=cmin,
  cmax=cmax)

To make our plot more brilliant, the axes are removed as follows:

In [81]:
noaxis=dict(showbackground=False,
  showgrid=False,
  showline=False,
  showticklabels=False,
  ticks='',
  title='',
  zeroline=False)

Figure title (‘3D spherical topography map’), font (‘Courier New’), and its associated settings are described here.
Note that, black and white are selected as the background and font color, respectively, to make our globe being in the universe:


In [82]:
import plotly.graph_objs as go

titlecolor = 'white'
bgcolor = 'black'

layout = go.Layout(
  autosize=False, width=1200, height=800,
  title = '3D spherical topography map',
  titlefont = dict(family='Courier New', color=titlecolor),
  showlegend = False,
  scene = dict(
    xaxis = noaxis,
    yaxis = noaxis,
    zaxis = noaxis,
    aspectmode='manual',
    aspectratio=go.layout.scene.Aspectratio(
      x=1, y=1, z=1)),
  paper_bgcolor = bgcolor,
  plot_bgcolor = bgcolor)

Finally, we create the plot and export as an HTML file.
Once you create the HTML file, you can check your plot using the Web browser (Google Chrome, Safari, …) in the local environment.

In [ ]:
from plotly.offline import plot

plot_data=[topo_sphere]
fig = go.Figure(data=plot_data, layout=layout)
plot(fig, validate = False, filename='SphericalTopography.html',
   auto_open=True)

In [ ]:
pwd


In the above plot, the topography data was displayed as the color without any three-dimensional shapes. Here we create those shapes by converting xs, ys, and zs. The following code describes how to create and make the figure just like what we did in the previous plot:

In [84]:
ratio_topo = 1.0 + topo*1e-5
xs_3d = xs*ratio_topo
ys_3d = ys*ratio_topo
zs_3d = zs*ratio_topo

topo_sphere_3d=dict(type='surface',
    x=xs_3d,
    y=ys_3d,
    z=zs_3d,
    colorscale=Ctopo,
    surfacecolor=topo,
    opacity=1.,
    cmin=cmin,
    cmax=cmax,
    showscale=False,
    hoverinfo='skip'
    )

plot_data_3DST=[topo_sphere_3d]
fig = go.Figure(data=plot_data_3DST, layout=layout)

fig.update_layout(title_text = '3D spherical topography map')
os.chdir(outpath)
plot(fig, validate = False, filename='3DSphericalTopography.html',
    auto_open=True)


NameError: name 'outpath' is not defined

In [85]:
os.getcwd()


'C:\\Users\\aozde'

2. Plot the Global Earthquake Distribution on the Interactive Globe
3 steps for achievement
2–1. Prepare the earthquake data
2–2. Prepare plate boundary plot and convert coordinates
2–3. Plot the earthquake distribution on the interactive globe created above
2–1. Prepare the earthquake data
Download the earthquake data
In this post, we download the earthquake data from the U.S. Geological Survey (USGS) website. This earthquake catalog data includes the dates, location parameters (longitude, latitude, and depth), magnitudes that describe the size of the earthquake, and so on. You can select the earthquake by filtering these parameters. I would recommend downloading only for larger magnitude events (e.g. M > 5) due to the huge numbers of earthquakes you have to handle (For a given frequency of magnitude 4.0 or larger events there will be 10 times as many magnitude 3.0 or larger quakes and 100 times as many magnitude 2.0 or larger quakes ! — Gutenberg-Richter law)
In this post, I downloaded the catalog based on the following options:
M > 5
Date between 2000/1/1 to 2020/8/31
Worldwide region


Import the earthquake data
Here we import the earthquake data downloaded above using pandas. And then create NumPy arrays for five parameters we will use (date, longitude, latitude, depth, magnitude). Three location parameters (longitude, latitude, and depth) are used as the x, y, and z, and magnitude is used for the plot size. For the plot color, we create two cases using depth, and Timefrom_RefYears (years since 2000) which is calculated using np.datetime64 and np.timedelta64 function.

In [ ]:
import pandas as pd
import glob

###############----For input----###############
file_list = glob.glob("data/query*.csv")
for i in range(len(file_list)):
    eachdata = pd.read_csv(file_list[i])
    if (i == 0):
        data = eachdata
    else:
        data = pd.concat([data, eachdata])

# Data selection
data = data[data.mag >= 5]        

# Change format to datetime for event date
data['np_DateTime']=pd.to_datetime(data['time'].str[:-2],format='%Y-%m-%dT%H:%M:%S')

date = np.array(data['np_DateTime'])
evlon = np.array(data['longitude'])
evlat = np.array(data['latitude'])
evDepth = np.array(data['depth'])
evMag = np.array(data['mag'])

# Calculate time difference
RefTime = np.datetime64("2000-01-01T00:00:00.000000000")
Timefrom_RefTime = (date - RefTime)
# For years
Timefrom_RefYears = Timefrom_RefTime.astype('timedelta64[Y]')
Timefrom_RefYears = Timefrom_RefYears / np.timedelta64(1, 'Y')


Convert the earthquake location to the spherical coordinates and create a color bar
Here we convert earthquake longitude and latitude to the spherical coordinates as described in 1–2, and create a color bar used for depth and Timefrom_RefYears. For a color bar, we adopt matplotlib color bar and convert to Plotly format.


In [ ]:
# Convert to spherical coordinates
xs_ev_org, ys_ev_org, zs_ev_org = mapping_map_to_sphere(evlon, evlat)

#Create color bar in Matplotlib
import matplotlib
from matplotlib import cm

def matplotlib_to_plotly(cmap, pl_entries):
    h = 1.0/(pl_entries-1)
    pl_colorscale = []
    
    for k in range(pl_entries):
        C = list(map(np.uint8, np.array(cmap(k*h)[:3])*255))
        pl_colorscale.append([k*h, 'rgb'+str((C[0], C[1], C[2]))])
        
    return pl_colorscale

def MlibCscale_to_Plotly(cbar):
    cmap = matplotlib.cm.get_cmap(cbar)
    rgb = []
    norm = matplotlib.colors.Normalize(vmin=0, vmax=255)

    for i in range(0, 255):
        k = matplotlib.colors.colorConverter.to_rgb(cmap(norm(i)))
        rgb.append(k)

    Cscale = matplotlib_to_plotly(cmap, 255)
    
    return Cscale

cbar = 'jet_r'
Cscale_EQ = MlibCscale_to_Plotly(cbar)


Create the earthquake distribution plot
Firstly, we slightly change the location parameters using depth to create a three-dimensional effect for the earthquake plot. Although earthquakes occur under the ground (of course!), we plot the earthquake distribution above the interactive globe to make them visible.

In [ ]:
# Create three-dimensional effect
ratio = 1. - evDepth*2e-4
xs_ev = xs_ev_org*ratio
ys_ev = ys_ev_org*ratio
zs_ev = zs_ev_org*ratio

ratio = 1.15 - evDepth*2e-4
xs_ev_up = xs_ev_org*ratio
ys_ev_up = ys_ev_org*ratio
zs_ev_up = zs_ev_org*ratio

2–2. Prepare the plate boundary plot and convert coordinates
In this post, we will plot the earthquake distribution on the interactive globe created in Step 1. However, in order to make an earthquake distribution more clear, we also plot this distribution on just a simple plate boundaries map. In this step, we prepare that plate boundary map. This procedure is already clearly explained in these posts:

In [ ]:
# Draw coast line
# Reference: https://plot.ly/~Dreamshot/9152

# Make shortcut to Basemap object, 
# not specifying projection type for this example
from mpl_toolkits.basemap import Basemap
m = Basemap(resolution='i') 

# Functions converting coastline/country polygons to lon/lat traces
def polygons_to_traces(poly_paths, N_poly):
    ''' 
    pos arg 1. (poly_paths): paths to polygons
    pos arg 2. (N_poly): number of polygon to convert
    '''
    # init. plotting list
    lons=[]
    lats=[]

    for i_poly in range(N_poly):
        poly_path = poly_paths[i_poly]
        
        # get the Basemap coordinates of each segment
        coords_cc = np.array(
            [(vertex[0],vertex[1]) 
             for (vertex,code) in poly_path.iter_segments(simplify=False)]
        )
        
        # convert coordinates to lon/lat by 'inverting' the Basemap projection
        lon_cc, lat_cc = m(coords_cc[:,0],coords_cc[:,1], inverse=True)
    
        
        lats.extend(lat_cc.tolist()+[None]) 
        lons.extend(lon_cc.tolist()+[None])
        
    return lons, lats

# Function generating coastline lon/lat 
def get_coastline_traces():
    poly_paths = m.drawcoastlines().get_paths() # coastline polygon paths
    N_poly = 91  # use only the 91st biggest coastlines (i.e. no rivers)
    cc_lons, cc_lats= polygons_to_traces(poly_paths, N_poly)
    return cc_lons, cc_lats

# Function generating country lon/lat 
def get_country_traces():
    poly_paths = m.drawcountries().get_paths() # country polygon paths
    N_poly = len(poly_paths)  # use all countries
    country_lons, country_lats= polygons_to_traces(poly_paths, N_poly)
    return country_lons, country_lats
  
# Get list of of coastline, country, and state lon/lat 
cc_lons, cc_lats=get_coastline_traces()
country_lons, country_lats=get_country_traces()

#concatenate the lon/lat for coastlines and country boundaries:
lons=cc_lons+[None]+country_lons
lats=cc_lats+[None]+country_lats

xs_bd, ys_bd, zs_bd = mapping_map_to_sphere(lons, lats, radius=1.01)# here the radius is slightly greater than 1 
                                                         #to ensure lines visibility; otherwise (with radius=1)
                                                         # some lines are hidden by contours colors
        
boundaries=dict(type='scatter3d',
               x=xs_bd,
               y=ys_bd,
               z=zs_bd,
               mode='lines',
               line=dict(color='gray', width=4)
              )

2–3. Plot the earthquake distribution on the interactive globe and the plate boundary plot created above
Using these new location parameters, we make the three different ways of Plotly function as follows:
3D earthquake distribution on the interactive globe with the color of depth
3D earthquake distribution on the interactive globe with the color of years since 2000
3D earthquake distribution on the plate boundary plot
3D earthquake distribution on the interactive globe with the color of depth
The color describes earthquake depths ranging from 0 (red) to 700 km (blue)


In [ ]:
depmax = 700.
depmin = 0.
depbin = 50.

cmin = depmin
cmax = depmax
cbin = depbin

seis_3D_depth_up = go.Scatter3d(x = xs_ev_up,
                      y = ys_ev_up,
                      z = zs_ev_up,
                      mode='markers',
                      name='measured',
                      marker = dict(
                          size = 1.*evMag,
                          cmax = cmax,
                          cmin = cmin,
                          colorbar = dict(
                              title = 'Source Depth',
                              titleside = 'right',
                              titlefont = dict(size = 16, 
                                               color = titlecolor, 
                                               family='Courier New'),
                              tickmode = 'array',
                              ticks = 'outside',
                              ticktext = list(np.arange(cmin,cmax+cbin,cbin)),
                              tickvals = list(np.arange(cmin,cmax+cbin,cbin)),
                              tickcolor = titlecolor,
                              tickfont = dict(size=14, color = titlecolor,
                                             family='Courier New')
                          ),
                          ### choose color option
                          color = evDepth,
                          ### choose color option
                          colorscale = Cscale_EQ,
                          showscale = True,
                          opacity=1.),
                      hoverinfo='skip'
                      )


3D earthquake distribution on the interactive globe with the color of years since 2000
The color describes how many years passed since 2000 from 0 year (red) to 20 years (blue)


In [ ]:
# Plot events in 3D with color of years since 2020
cmin = 0
cmax = 20
cbin = 1
seis_3D_time_up = go.Scatter3d(x = xs_ev_up,
                      y = ys_ev_up,
                      z = zs_ev_up,
                      mode='markers',
                      name='measured',
                      marker = dict(
                          size = 1.*evMag,
                          cmax = cmax,
                          cmin = cmin,
                          colorbar = dict(
                              title = 'Years since 2000',
                              titleside = 'right',
                              titlefont = dict(size = 16, 
                                               color = titlecolor,
                                               family='Courier New'),
                              tickmode = 'array',
                              ticks = 'outside',
                              ticktext = list(np.arange(cmin,cmax+cbin,cbin)),
                              tickvals = list(np.arange(cmin,cmax+cbin,cbin)),
                              tickcolor = titlecolor,
                              tickfont = dict(size=14, color = titlecolor,
                                             family='Courier New')
                          ),
                          ### choose color option
                          color = Timefrom_RefYears,
                          ### choose color option
                          colorscale = Cscale_EQ,
                          showscale = True,
                          opacity=1.),
                      hoverinfo='skip'
                      )

3D earthquake distribution on the plate boundary plot without interactive globe
Firstly, we create an inner black sphere to hide inside of the globe:

In [ ]:
ratio_k = 0.85
xs_k = xs * ratio_k
ys_k = ys * ratio_k
zs_k = zs * ratio_k

black_sphere=dict(type='surface',
    x=xs_k,
    y=ys_k,
    z=zs_k,
    colorscale=Cblack,
    surfacecolor=zs_k,
    opacity=1.,
    showscale=False,
    hoverinfo='skip'
    )

In [ ]:
Then create the 3D plot using the plate boundary data we prepared at 2.2:

In [ ]:
seis_3D_depth = go.Scatter3d(x = xs_ev,
                      y = ys_ev,
                      z = zs_ev,
                      mode='markers',
                      name='measured',
                      marker = dict(
                          size = 1.*evMag,
                          cmax = cmax,
                          cmin = cmin,
                          colorbar = dict(
                              title = 'Source Depth',
                              titleside = 'right',
                              titlefont = dict(size = 16, 
                                               color = titlecolor,
                                               family='Courier New'),
                              tickmode = 'array',
                              ticks = 'outside',
                              ticktext = list(np.arange(cmin,cmax+cbin,cbin)),
                              tickvals = list(np.arange(cmin,cmax+cbin,cbin)),
                              tickcolor = titlecolor,
                              tickfont = dict(size=14, color = titlecolor,
                                             family='Courier New')
                          ),
                          ### choose color option
                          color = evDepth,
                          ### choose color option
                          colorscale = Cscale_EQ,
                          showscale = True,
                          opacity=1.),
                      hoverinfo='skip'
                      )
